<a href="https://colab.research.google.com/github/nikshrimali/TSAI_END2_Phase1/blob/main/new_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install faiss
!pip install transformers
!sudo apt-get install libomp-dev


     |████████████████████████████████| 4.7 MB 5.6 MB/s 
     |████████████████████████████████| 2.9 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 47.9 MB/s 
     |████████████████████████████████| 3.3 MB 37.0 MB/s 
     |████████████████████████████████| 636 kB 37.7 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [

In [3]:
!git clone 'https://github.com/nikshrimali/TSAI_END2_Phase1.git'

Cloning into 'TSAI_END2_Phase1'...
remote: Enumerating objects: 936, done.
remote: Counting objects: 100% (936/936), done.
remote: Compressing objects: 100% (812/812), done.
remote: Total 936 (delta 178), reused 748 (delta 101), pack-reused 0
Receiving objects: 100% (936/936), 109.10 MiB | 25.06 MiB/s, done.
Resolving deltas: 100% (178/178), done.


In [4]:
from TSAI_END2_Phase1.utils import seed_all, get_device
from TSAI_END2_Phase1.models import BARTTrain, EnsembleTokens, Question_Model, GetEncodings, SearchSimilar, get_latest_checkpoint

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

device is cuda


In [5]:
device = get_device()
print(device)


Currently Running on cuda
cuda


In [6]:
import pandas as pd
import os
import torch


from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

df_context = pd.read_csv('TSAI_END2_Phase1/data/latest_data/groups_master.csv') 
df_merged = pd.read_csv('TSAI_END2_Phase1/data/latest_data/qna_gid.csv')
model_op = torch.load('TSAI_END2_Phase1/data/latest_data/encoded_context_348.pt')

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [12]:
# Inference
import glob
import torch

MODEL_STORE = os.path.join(os.getcwd(), '/content/drive/MyDrive/END_CAPSTONE')
bart_model = BARTTrain().to(device)
bart_model = get_latest_checkpoint('checkpoint', bart_model, MODEL_STORE)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Loading model from checkpoint checkpoint-3100
model successfully loaded checkpoint-3100


In [7]:

def get_latest_checkpoint(checkpoint, model, MODEL_STORE, q_string=''):

    checkpoints = sorted(glob.glob(f'{MODEL_STORE}/{checkpoint}*-[0-9]*'))
    if len(checkpoints):
        global_step = int(checkpoints[0].split('-')[-1])
        ckpt_name = '{}-{}'.format(checkpoint, global_step)
        print("Loading model from checkpoint %s" % ckpt_name)
        
        PATH = f'{MODEL_STORE}/{ckpt_name}/{checkpoint}{q_string}-{str(global_step)}.pt'
        # PATH = f'{MODEL_STORE}/{ckpt_name}/q_checkpoint-{str(global_step-1)}.pt'
        model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
        print("model successfully loaded %s" % ckpt_name)
    
    else:
        print("No checkpoints available right now")
    return model

In [ ]:
get_latest_checkpoint(checkpoint='qna_checkpoint', model=Question_Model, MODEL_STORE=MODEL_STORE, q_string='-q')

In [8]:
def inference(question, bart_tokenizer, bart_model, MODEL_STORE):

    # Get Pretrained BERT encodings
    ge = GetEncodings(MODEL_STORE = MODEL_STORE, type='questions')
    encoded_question = ge.encode(question, max_length=30)

    # Find top matching documents
    ss = SearchSimilar(iterator = df_context['context'].values.tolist(), filename='index.bin', embeddings=model_op, shape=768, device=device)
    similar_contexts = ss.get_n_similar_vectors(encoded_question, 3)
    similar_contexts.insert(0, question)

    combined_tokens = '</s></s>'.join(similar_contexts)

    print(f'Top similar document outputs is {combined_tokens}')

    # Prepare data for BART Inferencing

    source_encoding = tokenizer(
            combined_tokens,
            max_length=1024,
            padding='max_length',
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")
   

    # Inference BART Model
    output = bart_model(
            source_encoding['input_ids'].to(device),
            mode = 'eval')
    output = tokenizer.decode(output[0])
    print(output)
    return output


In [13]:
tokens = inference('What does torch.cosine loss do in pytorch?', tokenizer, bart_model, MODEL_STORE)


/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (

Loading model from checkpoint qna_checkpoint-4000
model successfully loaded qna_checkpoint-4000
Index file index.bin
Index trained - True
Top similar document outputs is What does torch.cosine loss do in pytorch?</s></s> torch.ne(). </s></s> torch.le(). </s></s>nonlinearity gain Linear / Identity 111 Conv{1,2,3}D 111 Sigmoid 111 Tanh 53\frac{5}{3}35​ ReLU 2\sqrt{2}2​ Leaky Relu 21+negative_slope2\sqrt{\frac{2}{1 + \text{negative\_slope}^2}}1+negative_slope22​​ SELU 34\frac{3}{4}43​ Warning 
Running model in eval mode
</s><s>TORTORTOR TOR TOR TORTORTORTorTorTor TOR TORTorTorTORTOR</s>
